# Train Model

In [ ]:
from torch import nn
import pickle
import torch.nn.functional as F
from src.Transformer import MyDataset_class
import torch
from torch.utils.data import DataLoader, random_split
import time
import os
from sklearn import metrics
import numpy as np
with open('params/peptide_vocab.pkl', 'rb') as f:
    w2i = pickle.load(f)
import random

def split_file(input_file, train_file, test_file, split_ratio=0.9):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    random.shuffle(lines)
    split_point = int(len(lines) * split_ratio)
    train_lines = lines[:split_point]
    test_lines = lines[split_point:]
    with open(train_file, 'w', encoding='utf-8') as f:
        f.writelines(train_lines)
    with open(test_file, 'w', encoding='utf-8') as f:
        f.writelines(test_lines)
os.makedirs(f"params/train",exist_ok=True)
os.makedirs(f"params/test",exist_ok=True)
split_file("params/pos_data", f"params/train/pos", f"params/test/pos")
split_file("params/neg_data", f"params/train/neg", f"params/test/neg")
split_file("params/neg_data_10", f"params/train/neg_10", f"params/test/neg_10")

In [ ]:
def train_model(model,save_path,negname,lr):
    num_epochs = 300
    os.makedirs(f"./{save_path}",exist_ok=True)
    os.makedirs(f"./{save_path}/model",exist_ok=True)
    all_data = MyDataset_class(f"params/train/pos",f"params/train/{negname}")
    train_num = int(len(all_data) * 0.85)
    val_num = int((len(all_data) - train_num))
    train_set, val_set = random_split(all_data,[train_num, val_num])
    train_loader = DataLoader(
        dataset=train_set,
        batch_size=512,
        shuffle=True,
        drop_last=True,
        num_workers=0,
        pin_memory=True
    )
    val_loader = DataLoader(
        dataset=val_set,
        batch_size=256,
        shuffle=True,
        drop_last=True,
        num_workers=0,
        pin_memory=True
    )
    print(f"Train set     : {train_num}")
    print(f"Val set       : {val_num}")
    print(f"Learning_rate : {lr}")
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = lr)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.99)
    best_acc_val = 0
    for epoch in range(num_epochs):
        model.train()
        losses = []
        start_time = time.time()
        for i, (data, labels) in enumerate(train_loader):
            output = model(data.cuda())
            loss = F.cross_entropy(output,labels.cuda())
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        scheduler.step()
        train_loss = np.array(losses).mean()
        model.eval()
        losses = []
        predict_all = np.array([], dtype=int)
        labels_all = np.array([], dtype=int)
        for i, (data, labels) in enumerate(val_loader):
            output = model(data.cuda())
            loss = F.cross_entropy(output,labels.cuda())
            losses.append(loss.item())
            predic = torch.max(output.data, 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)
        val_loss = np.array(losses).mean()
        val_acc = metrics.accuracy_score(labels_all,predict_all)
        val_report = metrics.classification_report(labels_all, predict_all, target_names=["AMP","NAMP"], digits=4)
        val_confusion = metrics.confusion_matrix(labels_all, predict_all)
        # torch.save(model.state_dict(),f"./{save_path}/model/{epoch}.pth")
        if val_acc > best_acc_val:
            best_acc_val = val_acc
            torch.save(model.state_dict(),f"./{save_path}/best_model.pth")
            with open(f"./{save_path}/model_data","w") as wf:
                wf.write(f"Epoch : {epoch}\nTrain loss : {train_loss}\nVal loss : {val_loss}\n")
                wf.write(f"************************  Val set  ************************\n")
                wf.write(f"{val_report}")
                wf.write("\n")
                wf.write(f"{val_confusion}")
                wf.write("\n")
        print(f"Epoch : {epoch}  Train loss : {train_loss}  Val loss : {val_loss}")

In [ ]:
from src.CNN import CNNModel
from src.RCNN import RCNNModel
from src.RNN_atten import RNN_attenModel
from src.RNN import RNNModel
from src.Transformer import TransformerModel, MyDataset_class

run_lr = [0.01,0.001,0.0001,0.00001]

for runlr in run_lr:
    model = CNNModel().cuda()
    train_model(model,f"Model_checkpoint-10/CNN_class_{runlr}","neg_10",runlr)
    model = CNNModel().cuda()
    train_model(model,f"Model_checkpoint-1/CNN_class_{runlr}","neg",runlr)

    model = RCNNModel().cuda()
    train_model(model,f"Model_checkpoint-10/RCNN_class_{runlr}","neg_10",runlr)
    model = RCNNModel().cuda()
    train_model(model,f"Model_checkpoint-1/RCNN_class_{runlr}","neg",runlr)

    model = RNN_attenModel().cuda()
    train_model(model,f"Model_checkpoint-10/RNN_atten_class_{runlr}","neg_10",runlr)
    model = RNN_attenModel().cuda()
    train_model(model,f"Model_checkpoint-1/RNN_atten_class_{runlr}","neg",runlr)

    model = RNNModel().cuda()
    train_model(model,f"Model_checkpoint-10/RNN_class_{runlr}","neg_10",runlr)
    model = RNNModel().cuda()
    train_model(model,f"Model_checkpoint-1/RNN_class_{runlr}","neg",runlr)

    model = TransformerModel().cuda()
    train_model(model,f"Model_checkpoint-10/Transformer_class_{runlr}","neg_10",runlr)
    model = TransformerModel().cuda()
    train_model(model,f"Model_checkpoint-1/Transformer_class_{runlr}","neg",runlr)

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from sklearn import metrics

def predict(data,model,predictor,vote=True):
    out = []
    if "CNN" in model:
        if vote:
            output = predictor["CNN"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:
            out.append(predictor["CNN"](data).unsqueeze(0))
    if "RCNN" in model:
        if vote:
            output = predictor["RCNN"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:
            out.append(predictor["RCNN"](data).unsqueeze(0))
    if "RNN" in model:
        if vote:
            output = predictor["RNN"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:        
            out.append(predictor["RNN"](data).unsqueeze(0))
    if "RNN_atten" in model:
        if vote:
            output = predictor["RNN_atten"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:        
            out.append(predictor["RNN_atten"](data).unsqueeze(0))
    if "Transformer" in model:
        if vote:
            output = predictor["Transformer"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:
            out.append(predictor["Transformer"](data).unsqueeze(0))
    if "CNN_10-fold" in model:
        if vote:
            output = predictor["CNN_10-fold"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:
            out.append(predictor["CNN_10-fold"](data).unsqueeze(0))
    if "RCNN_10-fold" in model:
        if vote:
            output = predictor["RCNN_10-fold"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:
            out.append(predictor["RCNN_10-fold"](data).unsqueeze(0))
    if "RNN_10-fold" in model:
        if vote:
            output = predictor["RNN_10-fold"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:        
            out.append(predictor["RNN_10-fold"](data).unsqueeze(0))
    if "RNN_atten_10-fold" in model:
        if vote:
            output = predictor["RNN_atten_10-fold"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:        
            out.append(predictor["RNN_atten_10-fold"](data).unsqueeze(0))
    if "Transformer_10-fold" in model:
        if vote:
            output = predictor["Transformer_10-fold"](data)
            predic = torch.max(output.data, 1)[1].cpu().unsqueeze(0)
            out.append(predic)
        else:
            out.append(predictor["Transformer_10-fold"](data).unsqueeze(0))
    if vote:
        out = (torch.concat(out).sum(0) > int(len(model)/2 -1)).int()
    else:
        out = torch.concat(out).mean(0)
        out = torch.max(out.data, 1)[1].cpu().unsqueeze(0).numpy()
    return out

## Change the best lr for each model

In [ ]:
best_run_lr = {
    "CNN":None,
    "RCNN":None,
    "RNN":None,
    "RNN_atten":None,
    "Transformer":None,
    "CNN_10-fold":None,
    "RCNN_10-fold":None,
    "RNN_10-fold":None,
    "RNN_atten_10-fold":None,
    "Transformer_10-fold":None,
}

In [ ]:
predictor = {
    "CNN":CNNModel(),
    "RCNN":RCNNModel(),
    "RNN":RNNModel(),
    "RNN_atten":RNN_attenModel(),
    "Transformer":TransformerModel(),
    "CNN_10-fold":CNNModel(),
    "RCNN_10-fold":RCNNModel(),
    "RNN_10-fold":RNNModel(),
    "RNN_atten_10-fold":RNN_attenModel(),
    "Transformer_10-fold":TransformerModel(),
}
for key in predictor.keys():
    if "_10-fold" in key:
        key_name = key.replace("_10-fold","")
        model_path = f"./Model_checkpoint-10/{key_name}_class_{best_run_lr[key]}/best_model.pth"
    else:
        key_name = key
        model_path = f"./Model_checkpoint-1/{key_name}_class_{best_run_lr[key]}/best_model.pth"
    predictor[key].load_state_dict(torch.load(model_path))
    predictor[key].cuda()
    predictor[key].eval()
allmodel = ["CNN","RCNN","RNN","RNN_atten","Transformer","CNN_10-fold","RCNN_10-fold","RNN_10-fold","RNN_atten_10-fold","Transformer_10-fold"]
test_set = MyDataset_class("params/test/pos","params/test/neg_10")
test_loader = DataLoader(
    dataset=test_set,
    batch_size=256,
    shuffle=True,
    drop_last=False,
    num_workers=0,
    pin_memory=True
)
from itertools import combinations

whethe_vote = [True,False,]
for vote in whethe_vote:
    best_acc = {
    "name":None,
    "acc":0,
    }
    best_auc = {
    "name":None,
    "auc":0,
    }
    best_amp_precision = {
    "name":None,
    "amp_precision":0,
    }
    with open(f"./combine_vote_{vote}","w") as wf:
        wf.write(f"combine,acc,auc,amp_precision,amp_recall,amp_f1-score,namp_precision,namp_recall,namp_f1-score\n")
        for i in range(2,11):
            for model in combinations(allmodel,i):
                predict_all = np.array([], dtype=int)
                labels_all = np.array([], dtype=int)
                for i, (data, labels) in enumerate(test_loader):
                    output = predict(data.cuda(),model,predictor,vote)
                    labels_all = np.append(labels_all, labels)
                    predict_all = np.append(predict_all, output)
                test_acc = metrics.accuracy_score(labels_all,predict_all)
                test_auc = metrics.roc_auc_score(labels_all,predict_all)
                test_report = metrics.classification_report(labels_all, predict_all, target_names=["AMP","NAMP"], digits=4)
                amp_list = test_report.split("\n")[2].split()
                namp_list = test_report.split("\n")[3].split()
                wf.write(f"{model},{test_acc},{test_auc},{amp_list[1]},{amp_list[2]},{amp_list[3]},{namp_list[1]},{namp_list[2]},{namp_list[3]}\n")
                if test_acc > best_acc["acc"]:
                    best_acc["acc"] = test_acc
                    best_acc["name"] = model
                if test_auc > best_auc["auc"]:
                    best_auc["auc"] = test_auc
                    best_auc["name"] = model
                if test_auc > float(amp_list[1]):
                    best_amp_precision["amp_precision"] = float(amp_list[1])
                    best_amp_precision["name"] = model
    print(f"******************************")
    print(vote)
    print(best_acc)
    print(best_auc)
    print(best_amp_precision)


# Filiter

In [ ]:
generate_seq_path = "pos_generate"
model = ['RCNN_10-fold', 'RNN_atten_10-fold', 'Transformer_10-fold']  #select best combine model

In [ ]:
from torch.utils.data import Dataset
from src.Transformer import peptide_tokenizer, encode_seq
import pandas as pd


class MyDataset_pred(Dataset):
    def __init__(self,file_path):
        super().__init__()
        self.data = pd.read_csv(file_path,header=None)
        self.num = len(self.data)
        self.data = self.data.values.tolist()
        self.max_length = 64
    def __len__(self):
        return  self.num
    def __getitem__(self, index):
        # AMP 0 NAMP 1
        seq = self.data[index]
        seq_list = peptide_tokenizer(seq[0])
        encoded_seq = encode_seq(seq_list, self.max_length - 1, w2i)
        encoded_seq = [0] + encoded_seq
        assert len(encoded_seq) == 64, print(seq)
        return torch.tensor(encoded_seq),seq[0]

predictor = {
    "CNN":CNNModel(),
    "RCNN":RCNNModel(),
    "RNN":RNNModel(),
    "RNN_atten":RNN_attenModel(),
    "Transformer":TransformerModel(),
    "CNN_10-fold":CNNModel(),
    "RCNN_10-fold":RCNNModel(),
    "RNN_10-fold":RNNModel(),
    "RNN_atten_10-fold":RNN_attenModel(),
    "Transformer_10-fold":TransformerModel(),
}
for key in predictor.keys():
    if "_10-fold" in key:
        key_name = key.replace("_10-fold","")
        model_path = f"./Model_checkpoint-10/{key_name}_class/best_model.pth"
    else:
        key_name = key
        model_path = f"./Model_checkpoint-1/{key_name}_class/best_model.pth"
    predictor[key].load_state_dict(torch.load(model_path))
    predictor[key].cuda()
    predictor[key].eval()

test_set = MyDataset_pred(generate_seq_path)
test_loader = DataLoader(
    dataset=test_set,
    batch_size=256,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    pin_memory=False
)
pos_seq = []
for i, (data, seqs) in enumerate(test_loader):
    output = predict(data.cuda(),model,predictor)
    indices = np.where(output == 0)[0]
    tem = np.array(seqs)[indices].tolist()
    # tem = np.array(seqs)[np.where(output == 0)].tolist()
    pos_seq.extend(tem)
pos_seq = list(set(pos_seq))
print(len(pos_seq))
with open(f"./pos_generate_filiter","w") as wf:
    for seq in pos_seq:
        wf.write(f"{seq}\n")

In [ ]:
def remove_konw(data, know_data):
    same_data = pd.merge(data, know_data, how="inner")
    outdata = data[~data.isin(same_data)].dropna()
    return outdata
def remove_c(data):
    data = data[~data[0].str.contains("C")]
    return data
def seq_check_pos_charge(seq):
    for i in range(len(seq)-4):
        tem_seq = seq[i:i+5]
        num = tem_seq.count("K") + tem_seq.count("R")
        if num > 3:
            return False
    return True
def remove_pos_charge(data):
    data = data[data[0].apply(seq_check_pos_charge)]
    return data
def seq_check_hydrophobic(seq):
    for i in range(len(seq)-2):
        tem_seq = seq[i:i+3]
        num = tem_seq.count("F") + tem_seq.count("V") + \
              tem_seq.count("I") + tem_seq.count("W") + \
              tem_seq.count("L") + tem_seq.count("A") + \
              tem_seq.count("M")
        if num == 3:
            return False
    return True
def remove_hydrophobic(data):
    data = data[data[0].apply(seq_check_hydrophobic)]
    return data
def seq_check_repeat_three(seq):
    for i in range(len(seq)-2):
        tem_seq = seq[i:i+3]
        # print(tem_seq)g 
        if tem_seq[0] == tem_seq[1] and tem_seq[0] == tem_seq[2]:
            return False
    return True
def remove_repeat_three(data):
    data = data[data[0].apply(seq_check_repeat_three)]
    return data


data = pd.read_csv(f"pos_generate_filiter",header=None)
know_data = pd.read_csv(f"params/pos_data",header=None)
data = remove_konw(data,know_data)
data = remove_c(data)
data = remove_pos_charge(data)
data = remove_hydrophobic(data)
data = remove_repeat_three(data)
data.to_csv(f"./finally_filiter",header= False,index = False)

In [ ]:
def convert_to_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        id_counter = 1
        for line in infile:
            sequence = line.strip()
            fasta_entry = f">seq_{id_counter}\n{sequence}\n"
            outfile.write(fasta_entry)
            id_counter += 1

input_file = 'finally_filiter'
output_file = 'finally_filiter.fasta'
convert_to_fasta(input_file, output_file)

# Cluster

In [ ]:
os.system(f"cd-hit -i finally_filiter.fasta -c 0.6 -o cluster_res -n 4")

# CGMD

## Run "python prepare_cgmd.py"

In [ ]:
import os
import shutil

source_folder = 'cgmd_train'
destination_folder = 'cgmd_train_pdb'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
for root, dirs, files in os.walk(source_folder):
    for file in files:
        if file.endswith('.pdb'):
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_folder, file)
            shutil.copy2(source_file_path, destination_file_path)

source_folder = 'cgmd_predict'
destination_folder = 'cgmd_predict_pdb'
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
for root, dirs, files in os.walk(source_folder):
    for file in files:
        if file.endswith('.pdb'):
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_folder, file)
            shutil.copy2(source_file_path, destination_file_path)

In [ ]:
import os
import shutil


file_list = os.listdir(f"./cgmd_predict_pdb") 
os.makedirs(f"./runmd_predict")
pmd = os.getcwd()
for file in file_list:
    file_path = f"{pmd}/cgmd_predict_pdb/{file}"
    os.makedirs(f"{pmd}/runmd_predict/{file}",exist_ok=True)
    shutil.copyfile(file_path,f"{pmd}/runmd_predict/{file}/peptide.pdb")
    os.system(f"{pmd}/martinize.py -f {pmd}/runmd_predict/{file}/peptide.pdb -o {pmd}/runmd_predict/{file}/system.top -x {pmd}/runmd_predict/{file}/peptide_cg.pdb -dssp /usr/bin/dssp -p backbone -ff martini22")
    shutil.copyfile(f"{pmd}/Protein.itp",f"./runmd_predict/{file}/Protein.itp")
    os.system(f"~/anaconda3/envs/py2.7/bin/python {pmd}/insane.py -f {pmd}/runmd_predict/{file}/peptide_cg.pdb -o {pmd}/runmd_predict/{file}/system.gro -p {pmd}/runmd_predict/{file}/system.top -pbc square -box 8,8,12 -sol W:90 -sol WF:10 -l POPC:3 -l POPG:1 -salt 0.15 -dm 4")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.2.itp",f"./runmd_predict/{file}/martini_v2.2.itp")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.0_POPG_02.itp",f"{pmd}/runmd_predict/{file}/martini_v2.0_POPG_02.itp")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.0_POPC_02.itp",f"{pmd}/runmd_predict/{file}/martini_v2.0_POPC_02.itp")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.0_ions.itp",f"{pmd}/runmd_predict/{file}/martini_v2.0_ions.itp")
    shutil.copyfile(f"{pmd}/mdp_file/equilibration.mdp",f"{pmd}/runmd_predict/{file}/equilibration.mdp")
    shutil.copyfile(f"{pmd}/mdp_file/minimization.mdp",f"{pmd}/runmd_predict/{file}/minimization.mdp")
    shutil.copyfile(f"{pmd}/mdp_file/production.mdp",f"{pmd}/runmd_predict/{file}/production.mdp")
    with open(f"{pmd}/runmd_predict/{file}/run.sh","w") as wf:
        wf.write(f"#!/bin/bash\n")
        wf.write(f"source ~/Software/gromacs/bin/GMXRC\n")
        wf.write(f"gmx grompp -f minimization.mdp -c system.gro -p system_run.top -o minimization.tpr\n")
        wf.write(f"gmx mdrun -deffnm minimization -v\n")
        wf.write(f"gmx grompp -f equilibration.mdp -c minimization.gro -p system_run.top -o equilibration.tpr -n index.ndx\n")
        wf.write(f"gmx mdrun -deffnm equilibration -v\n")
        wf.write(f"gmx grompp -f production.mdp -c equilibration.gro -p system_run.top -o dynamic.tpr -n index.ndx\n")
        wf.write(f"gmx mdrun -deffnm dynamic -v\n")

    with open(f"{pmd}/runmd_predict/{file}/system.top") as rf:
        lines = rf.readlines()
        with open(f"{pmd}/runmd_predict/{file}/system_run.top","w") as wf:
            wf.write(f"#include \"martini_v2.2.itp\"\n")
            wf.write(f"#include \"martini_v2.0_POPC_02.itp\"\n")
            wf.write(f"#include \"martini_v2.0_POPG_02.itp\"\n")
            wf.write(f"#include \"martini_v2.0_ions.itp\"\n")
            wf.write(f"#include \"Protein.itp\"\n")

            for i, line in enumerate(lines):
                if i == 0:
                    continue
                wf.write(line)
    pop = os.popen(f"gmx make_ndx -f {pmd}/runmd_predict/{file}/system.gro -o {pmd}/runmd_predict/{file}/index.ndx\n","w")
    
    pop.write(f"13|14\n")
    pop.write(f"15|16|19\n")
    pop.write(f"q\n")
    pop.close()

In [ ]:
import os
import shutil


file_list = os.listdir(f"./cgmd_train_pdb") 
os.makedirs(f"./runmd")
pmd = os.getcwd()
for file in file_list:
    file_path = f"{pmd}/cgmd_train_pdb/{file}"
    os.makedirs(f"{pmd}/runmd/{file}",exist_ok=True)
    shutil.copyfile(file_path,f"{pmd}/runmd/{file}/peptide.pdb")
    os.system(f"{pmd}/martinize.py -f {pmd}/runmd/{file}/peptide.pdb -o {pmd}/runmd/{file}/system.top -x {pmd}/runmd/{file}/peptide_cg.pdb -dssp /usr/bin/dssp -p backbone -ff martini22")
    shutil.copyfile(f"{pmd}/Protein.itp",f"./runmd/{file}/Protein.itp")
    os.system(f"/home/wang/anaconda3/envs/py2.7/bin/python {pmd}/insane.py -f {pmd}/runmd/{file}/peptide_cg.pdb -o {pmd}/runmd/{file}/system.gro -p {pmd}/runmd/{file}/system.top -pbc square -box 8,8,12 -sol W:90 -sol WF:10 -l POPC:3 -l POPG:1 -salt 0.15 -dm 4")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.2.itp",f"./runmd/{file}/martini_v2.2.itp")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.0_POPG_02.itp",f"{pmd}/runmd/{file}/martini_v2.0_POPG_02.itp")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.0_POPC_02.itp",f"{pmd}/runmd/{file}/martini_v2.0_POPC_02.itp")
    shutil.copyfile(f"{pmd}/md_test/martini_v2.0_ions.itp",f"{pmd}/runmd/{file}/martini_v2.0_ions.itp")
    shutil.copyfile(f"{pmd}/mdp_file/equilibration.mdp",f"{pmd}/runmd/{file}/equilibration.mdp")
    shutil.copyfile(f"{pmd}/mdp_file/minimization.mdp",f"{pmd}/runmd/{file}/minimization.mdp")
    shutil.copyfile(f"{pmd}/mdp_file/production.mdp",f"{pmd}/runmd/{file}/production.mdp")
    with open(f"{pmd}/runmd/{file}/run.sh","w") as wf:
        wf.write(f"#!/bin/bash\n")
        wf.write(f"source ~/Software/gromacs/bin/GMXRC\n")
        wf.write(f"gmx grompp -f minimization.mdp -c system.gro -p system_run.top -o minimization.tpr\n")
        wf.write(f"gmx mdrun -deffnm minimization -v\n")
        wf.write(f"gmx grompp -f equilibration.mdp -c minimization.gro -p system_run.top -o equilibration.tpr -n index.ndx\n")
        wf.write(f"gmx mdrun -deffnm equilibration -v\n")
        wf.write(f"gmx grompp -f production.mdp -c equilibration.gro -p system_run.top -o dynamic.tpr -n index.ndx\n")
        wf.write(f"gmx mdrun -deffnm dynamic -v\n")

    with open(f"{pmd}/runmd/{file}/system.top") as rf:
        lines = rf.readlines()
        with open(f"{pmd}/runmd/{file}/system_run.top","w") as wf:
            wf.write(f"#include \"martini_v2.2.itp\"\n")
            wf.write(f"#include \"martini_v2.0_POPC_02.itp\"\n")
            wf.write(f"#include \"martini_v2.0_POPG_02.itp\"\n")
            wf.write(f"#include \"martini_v2.0_ions.itp\"\n")
            wf.write(f"#include \"Protein.itp\"\n")

            for i, line in enumerate(lines):
                if i == 0:
                    continue
                wf.write(line)
    pop = os.popen(f"gmx make_ndx -f {pmd}/runmd/{file}/system.gro -o {pmd}/runmd/{file}/index.ndx\n","w")
    
    pop.write(f"13|14\n")
    pop.write(f"15|16|19\n")
    pop.write(f"q\n")
    pop.close()
os.chdir(pmd)

Run CGMD

In [ ]:
file_name = "runmd"
file_list = os.listdir(f"./{file_name}")
path = os.getcwd()
for file in file_list:
    if os.path.exists(f"{path}/{file_name}/{file}/have_run"):
        print(f"{path}/{file_name}/{file} have Run!")
        continue
    else:
        os.mkdir(f"{path}/{file_name}/{file}/have_run")
        os.chdir(f"{path}/{file_name}/{file}")
        os.system(f"sh run.sh")
        print(f"****************************")
        print(f"MDRUN : {path}/runmd/{file}")
file_name = "runmd_predict"
file_list = os.listdir(f"./{file_name}")
path = os.getcwd()
for file in file_list:
    if os.path.exists(f"{path}/{file_name}/{file}/have_run"):
        print(f"{path}/{file_name}/{file} have Run!")
        continue
    else:
        os.mkdir(f"{path}/{file_name}/{file}/have_run")
        os.chdir(f"{path}/{file_name}/{file}")
        os.system(f"sh run.sh")
        print(f"****************************")
        print(f"MDRUN : {path}/runmd/{file}")
os.chdir(path)

In [ ]:
import os

path = os.getcwd()
floder_list = os.listdir(f"{path}/runmd")
for floder in floder_list:
    floder_path = f"{path}/runmd/{floder}"
    os.chdir(floder_path)
    try:
        os.makedirs(f"{path}/runmd/{floder}/whole")
        pop = os.popen(f"gmx trjconv -s dynamic.tpr -f dynamic.xtc -o dynamic_whole.xtc -pbc whole","w")
        pop.write(f"0\n")
        pop.close()
    except:
        pass

floder_list = os.listdir(f"{path}/runmd_predict")
for floder in floder_list:
    floder_path = f"{path}/runmd_predict/{floder}"
    os.chdir(floder_path)
    try:
        os.makedirs(f"{path}/runmd_predict/{floder}/whole")
        pop = os.popen(f"gmx trjconv -s dynamic.tpr -f dynamic.xtc -o dynamic_whole.xtc -pbc whole","w")
        pop.write(f"0\n")
        pop.close()
    except:
        pass

In [ ]:
from MDAnalysis import Universe
from MDAnalysis.analysis import distances
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def get_line(gro_path,xtc_path,save_path_data):
    product = Universe(gro_path,xtc_path,)
    peptide = product.select_atoms(f"protein")
    member_heads = product.select_atoms(f"resname POPC or resname POPG")
    all_list = []
    for ts in product.trajectory:
        out = distances.distance_array(peptide,member_heads)
        out_numpy = np.array(out).min(-1)
        all_list.append(out_numpy.tolist())
    data = np.array(all_list)
    name = []
    for atom in peptide:
        name.append(f"{atom.resname}_{atom.id}")
    data_T = data.T
    np.save(save_path_data,data_T)

os.makedirs(f"./out_whole",exist_ok=True)
os.makedirs(f"./out_whole/data",exist_ok=True)

floder_list = os.listdir(f"./runmd")
path = os.getcwd()
for floder in tqdm(floder_list):
    floder_path = f"{path}/runmd/{floder}"
    try:
        os.makedirs(f"{path}/runmd/{floder}/analysis_whole")
        print(f"analysis {floder}")
        get_line(f"{floder_path}/dynamic.gro",f"{floder_path}/dynamic_whole.xtc",f"./out_whole/data/{floder}.npy")
    except:
        pass

os.makedirs(f"./predict",exist_ok=True)
os.makedirs(f"./predict/data",exist_ok=True)

floder_list = os.listdir(f"./runmd_predict")
path = os.getcwd()
for floder in tqdm(floder_list):
    floder_path = f"{path}/runmd_predict/{floder}"
    try:
        os.makedirs(f"{path}/runmd_predict/{floder}/analysis_whole")
        print(f"analysis {floder}")
        get_line(f"{floder_path}/dynamic.gro",f"{floder_path}/dynamic_whole.xtc",f"./predict/data/{floder}.npy")
    except:
        pass



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics


data = []
label = []
file_list = os.listdir(f"./out/data")
for file in file_list:
    temdata = np.load(f"./out/data/{file}").mean(0).tolist()
    data.append(temdata)
    aname = file.split(".")[0]
    if "pos" in aname:
        label.append(0)
    else:
        label.append(1)
data = np.array(data)
label = np.array(label)
X_train,X_test,y_train,y_test = train_test_split(data,label,test_size=0.2)
rfc = RandomForestClassifier()

parameters = {'n_estimators': range(50,500,50),'max_depth':range(3,15,2),
                'min_samples_leaf':[5,6,7],'max_features':[1,2,3,4,5,6]}
grid_rfc = GridSearchCV(rfc,parameters,scoring='f1_macro',n_jobs=-1)
grid_rfc.fit(X_train,y_train)
print(grid_rfc.best_params_,grid_rfc.best_score_)

In [ ]:
rfc = RandomForestClassifier(n_estimators=50,max_depth=9,max_features=5,min_samples_leaf=7,bootstrap=True,criterion="entropy")
rfc.fit(X_train,y_train)
pred = rfc.predict(X_test)

In [ ]:
file_list = os.listdir(f"./out_generate/data")
pred_proba = []
pred_name = []
with open(f"./filiter_out_new","w") as wf:
    for file in file_list:
        adata = np.load(f"./out_generate/data/{file}").mean(0)
        adata = np.expand_dims(adata,axis=0)
        if adata.shape[1] != 25001:
            print(file)
            continue
        wf.write(f"{file}\t{rfc.predict_proba(adata)[:,0]}\n")
        pred_proba.append(rfc.predict_proba(adata)[:,0])
        pred_name.append(file)
pred_proba_np = np.array(pred_proba).squeeze()
argsort_np = pred_proba_np.argsort()
with open(f"./filiter_out_new_sort","w") as wf:
    for i in argsort_np:
        wf.write(f"{pred_name[i]}\t\t{pred_proba[i][0]}\n")